In [ ]:
# 변수 설정
VBP = "VBP"
VBN = "VBN"
VBG = "VBG"
VBD = "VBD"
VBZ = "VBZ"
RBR = "RBR"
RB = "RB"
RBS = "RBS"
NN = "NN"
NNS = "NNS"
JJ = "JJ"
JJS = "JJS"
JJR = "JJR"
TO_VERB = "TO_VERB"
DT = 'DT'

In [ ]:
import pandas as pd

problem_df = pd.read_csv('어법 문제.csv', index_col=0)
problem_df = problem_df.dropna()
problem_df['정답 text']

s = problem_df.iloc[3,6]
s  

'One way for a salesperson to gather knowledge about competitors is to study available literature and bulletins published on competitive products. These bulletins can usually be obtained at trade shows. Additional facts can be gathered by reading trade journals and competitors’ advertisements. It may be helpful for the salesperson to cut out and save each ad run by the competitors, because these ads reveal what products they are pushing. These ads will also reveal what features and benefits they are promoting on their products. Having this information enables a salesperson to make counter pitches. A salesperson can also compare competitive ads and bulletins with his company’s material and perhaps assist the advertising department in improving the company’s brochures and advertising.'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

어법 문제 Generate 함수

In [ ]:
from nltk.tokenize import sent_tokenize
import random
from nltk.tokenize.treebank import TreebankWordDetokenizer



def generate_problem(paragraph, question_type) :
  nlp = spacy.load('en_core_web_sm')

  sentences = [(sent, idx) for idx, sent in enumerate(sent_tokenize(paragraph))] # 문장 전체 list

  if question_type == 1:  # 1,2,3,4,5 중에 틀린거 하나
    trans_num = 5     
    answer_type = ['①', '②', '③', '④', '⑤']   
  else:  # ABC
    trans_num = 3
    answer_type = ['(A)', '(B)', '(C)']   


  # 바꿀 문장 개수(문제 형태 유형)에 따라 바꿀 대상 선정 
  trans_before = random.sample(sentences, trans_num) #바꿀 대상 선정
  trans_after = []
  new_sentences = [sent[0] for sent in sentences]
  
  # pos mapping
  # TO_VERB : 분사 앞에 be동사가 있을 경우 be동사에 맞게 수일치, 시제 맞춰서 변환시켜야 함. 
  pos_trans_dict = {'VBP': ['VBN', 'VBZ', 'VBD'], 'VBG' : ['TO_VERB', 'VBN'],  
                    'VBN':['TO_VERB', 'VBG'], 'VBZ':['VBN', 'VBZ', 'VBD'], 
                    'NN':['NNS'], 'NNS':['NN'], 
                    'DT':[{"this":"these", "these":'this', "that":'those', 'those':'that'}]}
                        
                        
                        
                         #'RB':['JJ'], 'RBR':['JJR'], 'RBS':['JJS'], 
  determiner_transform_dict = {
      "this": ["these", "that", "those"],
      "these": ["this", "those","that"],
      "those":["that","this","these"],
      "another": ["one"],
      "some":["another"],
      }
  determiner_transform_dict_small = {"this":["these"], "these":['this'], "that":['those'], 'those':['that']}
  be_verb = ['be', 'was', 'were', 'being', 'been', 'am', 'are', 'is']
  answers = []
  answer_return = {}
  #  {"A": {"answer": "is", "wrong_answer": "was"},...}
  # convert sentence
  
  print(trans_before)
  trans_before = sorted(trans_before, key=lambda x: x[1])
  print(trans_before)

  nlp = spacy.load('en_core_web_sm')
  for trans_idx, sent_obj in enumerate(trans_before) :
    # 우리가 변환 가능한 pos를 가지고 있는 단어들만 모으기
    sent_idx = sent_obj[1]
    sent = sent_obj[0]
    tokens = nlp(sent)
    avail_tokens = []
    new_tokens = [token.text for token in tokens]
    for idx,token in enumerate(tokens):
      # print("sentence" , idx)
      tag = token.tag_
      # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
      

      if(tag in pos_trans_dict.keys()):
        # 지시대명사의 경우 따로 처리 
        if tag=='DT' and not token.text in pos_trans_dict['DT'][0].keys() :
          print(token.text)
          pass
        else :
          if(tokens[idx-1]):
            prev_tokens = tokens[idx-1]
          else:
            prev_tokens = None
          avail_tokens.append([token, idx, prev_tokens])
    
    while True:
      print(avail_tokens)
      src_token_obj = random.choice(avail_tokens)
      src_token = src_token_obj[0]
      src_tag = src_token.tag_
      print(src_token_obj)

      # DT의 경우 
      if src_tag == "DT":
        target_pos = pos_trans_dict["DT"][0][src_token.text]
        answer = chg_pos(src_token_obj, target_pos)
        if(answer):
          pos_trans_dict.pop('DT')
          answers.append((src_token_obj, answer))
          break
        else:
          continue

      else :
        target_pos = random.choice(pos_trans_dict[src_tag])
        answer = chg_pos(src_token_obj, target_pos)
        if(answer):
          print(src_tag, target_pos)
          pos_trans_dict[src_tag].remove(target_pos)
          if(len(pos_trans_dict[src_tag]) == 0):
            pos_trans_dict.pop(src_tag)
          answers.append((src_token_obj, answer))
          break 
        else:
          continue


      #elif src_tag == "VBG" or src_tag == "VBN"
    answer = answers[trans_idx]
    print("answer print")
    print(answer[0][0].text)
    print(answer[1])
    print(type(tokens))
    new_tokens[answer[0][1]] = answer_type[trans_idx] + "[" + answer[0][0].text + '/' + answer[1] + "]"
    new_sen = TreebankWordDetokenizer().detokenize(new_tokens)
    new_sentences[sent_idx] = new_sen
    answer_dict = {'answer': answer[0][0].text, 'wrong_answer':answer[1] }
    answer_return[answer_type[trans_idx]] = answer_dict
    
   
   
  return ' '.join(new_sentences), answer_return
  

pos 태그 변환 함수

In [ ]:
pos_trans_dict = {'VBP': ['VBN', 'VBZ', 'VBD'], 'VBG' : ['TO_VERB', 'VBN'],  'VBN':['TO_VERB', 'VBG'], 'VBZ':['VBN', 'VBZ', 'VBD'], 
                         'NN':['NNS'], 'NNS':['NN'], 'DT':[{"this":"these", "these":'this', "that":'those', 'those':'that'}]}
pos_det_dict = {"this":"these", "these":'this', "that":'those', 'those':'that'} #'RB':['JJ'], 'RBR':['JJR'], 'RBS':['JJS'],
be_verb = ['be', 'was', 'were', 'being', 'been', 'am', 'are', 'is']
import random
import spacy
import pyinflect
def chg_pos(src_token_obj, target_pos) : 
  nlp = spacy.load('en_core_web_sm')
  src_token = src_token_obj[0]
  src_text = src_token.text
  src_tag = src_token.tag_
  print(src_tag, target_pos)
  target_token = ''
  print("chg_pos")
  if(src_tag == DT) :
    target_token = pos_det_dict[src_text]
  elif(target_pos == TO_VERB) :
    prev_token_pos = src_token_obj[2].pos_
    target_token = src_token._.inflect(prev_token_pos)
  elif(src_tag == VBP and target_pos == VBN):
    if(src_text in be_verb):  
      target_token = None
    else :
      target_token = 'are' + src_token._.inflect(target_pos)
  elif(src_tag in [RB, RBR, RBS]):
    try:
      src_lemma = src_token.lemma_
      src_all = getAllInflections(src_lemma, pos_type='A')
      target_token = src_all[target_pos][0]
    except:
      traget_token = None
  else:
    target_token = src_token._.inflect(target_pos)

  if(src_text == target_token):
    target_token = None

  return target_token
    
  
  
  

  

In [ ]:
generate_problem(s,2)